# VIVOS TTS Training Notebook

Notebook này giúp huấn luyện mô hình TTS sử dụng dữ liệu VIVOS trên Google Colab với GPU.

## Cài đặt và chuẩn bị môi trường

Đầu tiên, clone repository từ GitHub và cài đặt các dependencies cần thiết.

In [ ]:
# Kiểm tra GPU
!nvidia-smi

In [ ]:
# Clone repository từ GitHub
!git clone https://github.com/DangAW2002/vietnamese-tts.git
%cd vietnamese-tts

In [ ]:
# Cài đặt các package cần thiết
!pip install torch torchaudio numpy==2.0.0 numba<0.58.0 scipy librosa==0.10.1 matplotlib pandas tensorboard tqdm pyyaml soundfile gdown

## Tải dữ liệu đã được xử lý

Do kích thước lớn, dữ liệu VIVOS đã xử lý được lưu trữ riêng trên Google Drive. Chúng ta cần tải và giải nén trước khi sử dụng.

In [ ]:
# Tải dữ liệu đã xử lý từ Google Drive
import os
import gdown
import zipfile
from pathlib import Path

# Tạo thư mục data
os.makedirs("data/processed", exist_ok=True)

# Tạo script để upload dữ liệu lên Google Drive nếu chưa có
if not os.path.exists("upload_data.py"):
    with open("upload_data.py", "w") as f:
        f.write("""
# Script này giúp upload dữ liệu đã xử lý lên Google Drive
from google.colab import drive
import os
import zipfile

# Mount Google Drive
drive.mount('/content/drive')

# Nén thư mục dữ liệu đã xử lý
print("Đang nén dữ liệu processed...")
with zipfile.ZipFile('vivos_processed.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk('data/processed'):
        for file in files:
            zipf.write(os.path.join(root, file))

# Copy file nén vào Google Drive
print("Đang copy dữ liệu vào Drive...")
os.system('cp vivos_processed.zip "/content/drive/My Drive/"')
print("Hoàn thành! File vivos_processed.zip đã được lưu vào Drive.")
""")

# Thông báo hướng dẫn cho người dùng
print("="*80)
print("HƯỚNG DẪN: Bạn cần upload dữ liệu VIVOS đã xử lý lên Google Drive của bạn")
print("1. Nếu bạn đã xử lý dữ liệu trên máy, hãy chạy script upload_data.py để upload lên Drive")
print("2. Sau đó lấy share link của file vivos_processed.zip từ Drive")
print("3. Cập nhật URL bên dưới với share link của bạn")
print("="*80)

# URL của file zip chứa dữ liệu đã xử lý (cần thay thế bằng link Google Drive thật)
url = "PASTE_YOUR_GDRIVE_LINK_HERE"

# Hỏi người dùng có muốn tải xuống không
download_data = input("Bạn đã cập nhật URL và muốn tải dữ liệu xuống? (yes/no): ")
if download_data.lower() in ["yes", "y"]:
    # Tải file zip
    zip_path = "vivos_processed.zip"
    print("Đang tải dữ liệu VIVOS đã xử lý...")
    gdown.download(url, zip_path, quiet=False)
    
    # Giải nén
    print("Đang giải nén dữ liệu...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("data/processed")
    
    # Xóa file zip
    os.remove(zip_path)
    print("Hoàn thành tải và giải nén dữ liệu!")
else:
    print("Bỏ qua bước tải dữ liệu. Hãy đảm bảo bạn đã có dữ liệu trong thư mục data/processed.")

## Huấn luyện mô hình

Bây giờ chúng ta sẽ huấn luyện mô hình TTS sử dụng dữ liệu VIVOS đã được tiền xử lý.

In [ ]:
# Kiểm tra dữ liệu đã được tải đúng cách chưa
import os
import glob

train_files = glob.glob("data/processed/train/*.pt")
val_files = glob.glob("data/processed/val/*.pt")
test_files = glob.glob("data/processed/test/*.pt")

print(f"Số file trong tập train: {len(train_files)}")
print(f"Số file trong tập val: {len(val_files)}")
print(f"Số file trong tập test: {len(test_files)}")

if len(train_files) == 0 or len(val_files) == 0 or len(test_files) == 0:
    print("CẢNH BÁO: Không tìm thấy dữ liệu đã xử lý. Hãy quay lại bước trước để tải dữ liệu.")

In [ ]:
# Chạy script training
# Các tham số có thể điều chỉnh:
# --epochs: số epoch
# --batch_size: kích thước batch
# --lr: learning rate

!python src/colab_train.py --epochs 50 --batch_size 64

## Hiển thị quá trình huấn luyện với TensorBoard

In [ ]:
# Cài đặt TensorBoard extension
%load_ext tensorboard
%tensorboard --logdir=checkpoints/logs

## Lưu mô hình đã huấn luyện lên Google Drive

Sau khi huấn luyện xong, bạn nên lưu mô hình lên Google Drive để tránh mất kết quả khi phiên Colab kết thúc.

In [ ]:
# Mount Google Drive để lưu mô hình
from google.colab import drive
drive.mount('/content/drive')

# Nén thư mục checkpoints
!zip -r checkpoints.zip checkpoints/

# Copy file nén vào Google Drive
import os
import shutil

# Tạo thư mục để lưu trữ mô hình trên Drive nếu chưa có
os.makedirs("/content/drive/My Drive/vietnamese-tts-models", exist_ok=True)

# Copy file zip vào Drive
shutil.copy("checkpoints.zip", "/content/drive/My Drive/vietnamese-tts-models/")
print("Đã lưu checkpoints.zip vào Google Drive thành công!")